In [40]:
from splinter import Browser
from flask import Flask, jsonify, redirect
import pandas as pd
from bs4 import BeautifulSoup as bs
import time

In [41]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

#### https://mars.nasa.gov/news/

In [34]:
# finding news title and teaser
browser.visit("https://mars.nasa.gov/news/")
time.sleep(2)
soup = bs(browser.html, "html.parser")

news_titles = []
news_paragraphs = []

items = soup.find("ul",class_="item_list")
slides = items.find_all("li", class_="slide")
for slide in slides:
    news_title = slide.find("div", class_="content_title").text
    news_p = slide.find("div", class_="article_teaser_body").text
    news_titles.append(news_title)
    news_paragraphs.append(news_p)
    
print(f"Title Example: {news_titles[0]}")
print(f"Paragraph Example: {news_paragraphs[0]}")

Title Example: NASA Readies Perseverance Mars Rover's Earthly Twin 
Paragraph Example: Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


#### https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

In [35]:
# finding featured image url
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
browser.find_by_id("full_image").click()
time.sleep(2)

soup = bs(browser.html, "html.parser")
image_src = soup.find("img", class_="fancybox-image")["src"]

featured_image_url = f"https://jpl.nasa.gov{image_src}"
featured_image_url

'https://jpl.nasa.gov/spaceimages/images/mediumsize/PIA13911_ip.jpg'

#### https://space-facts.com/mars/

In [36]:
# getting mars facts
browser.visit("https://space-facts.com/mars/")
df = pd.read_html(browser.html)[1]
df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [37]:
result = df.to_html(index=False).strip()
result

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

#### https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

In [38]:
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
time.sleep(1)
soup = bs(browser.html, "html.parser")

# retrieving hemishere urls
hemisphere_urls = []
hemispheres = soup.find_all("div", class_="description")
for hemisphere in hemispheres:
    url = hemisphere.find("a")["href"]
    url = f"https://astrogeology.usgs.gov{url}"
    hemisphere_urls.append(url)

# retrieving titles and links
results = []
for hemisphere_url in hemisphere_urls:
    browser.visit(hemisphere_url)
    time.sleep(2)
    soup = bs(browser.html, "html.parser")
    title = soup.find("h2", class_="title").text
    image_url = soup.find_all("li")[0].find("a")["href"]
    results.append({"title":title, "image_url":image_url})

results

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

In [39]:
browser.quit()